In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST 손글씨 이미지 데이터 읽어들이기
mnist = input_data.read_data_sets("mnist/", one_hot=True)

pixels = 28*28 #28x28 픽셀
nums = 10 #0-9사이의 카테고리

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

In [3]:
#플레이스홀더 정의하기
x = tf.placeholder(tf.float32, shape = (None, pixels), name="x") #이미지 데이터
y_ = tf.placeholder(tf.float32, shape = (None, nums), name="y_") #정답 레이블

In [8]:
#가중치와 바이어스를 초기화하는 함수
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="W_"+name)
    return W

def bias_variable(name, size):
    b_init = tf.constant(0.1, shape=[size])
    b= tf.Variable(b_init, name="b_"+name)
    return b

In [5]:
#합성곱 계층을 만드는 함수
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding='SAME')

In [6]:
#최대 풀링층을 만드는 함수
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

In [9]:
#합성곱층1
with tf.name_scope('conv1')as scope:
    W_conv1 = weight_variable('conv1', [5,5,1,32])
    b_conv1 = bias_variable('conv1', 32)
    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)

In [10]:
#풀링층1
with tf.name_scope('pool1')as scope:
    h_pool1 = max_pool(h_conv1)

In [11]:
#합성곱층2
with tf.name_scope('conv2')as scope:
    W_conv2 = weight_variable('conv2', [5,5,32,64])
    b_conv2 = bias_variable('conv2', 64)
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)

In [12]:
#풀링층2
with tf.name_scope('pool2')as scope:
    h_pool2 = max_pool(h_conv2)

In [13]:
#전결합층
with tf.name_scope('fully_connected')as scope:
    n = 7*7*64
    W_fc = weight_variable('fc', [n, 1024])
    b_fc = bias_variable('fc', 1024)
    h_pool2_flat = tf.reshape(h_pool2, [-1,n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc)+b_fc)

In [14]:
#드롭아웃(오버피팅) 막기
with tf.name_scope('dropout')as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
#출력층
with tf.name_scope('readout')as scope:
    W_fc2 = weight_variable('fc2', [1024,10])
    b_fc2 = bias_variable('fc2', 10)
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2)+b_fc2)

In [16]:
#모델 학습시키기
with tf.name_scope('loss')as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
with tf.name_scope('training')as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entropy)

In [22]:
#모델 평가하기
with tf.name_scope('predict')as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))

In [18]:
#feed_dict 설정하기
def set_feed(images, labels, prob):
    return {x: images, y_ : labels, keep_prob:prob}

In [23]:
#세션 시작하기
with tf.Session()as sess:
    sess.run(tf.global_variables_initializer())
    #TensorBoard 준비하기
    tw = tf.summary.FileWriter('log_dir', graph=sess.graph)
    #테스트 전용 피드 만들기
    test_fd = set_feed(mnist.test.images, mnist.test.labels, 1)
    #학습 시작하기
    for step in range(10000):
        batch = mnist.train.next_batch(50)
        fd = set_feed(batch[0], batch[1], 0.5)
        _, loss = sess.run([train_step, cross_entropy], feed_dict=fd)
        if step % 100 ==0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step=", step, "loss=", loss, "acc=", acc)
     #최종적인 결과 출력하기
    acc=sess.run(accuracy_step, feed_dict=test_fd)
    print("정답률 =", acc)

step= 0 loss= 477.4867 acc= 0.1711
step= 100 loss= 49.622566 acc= 0.849
step= 200 loss= 45.811306 acc= 0.9152
step= 300 loss= 23.477467 acc= 0.9284
step= 400 loss= 11.511037 acc= 0.9394
step= 500 loss= 11.193548 acc= 0.9465
step= 600 loss= 14.064208 acc= 0.9512
step= 700 loss= 9.548792 acc= 0.9569
step= 800 loss= 14.98844 acc= 0.9588
step= 900 loss= 8.564086 acc= 0.9642
step= 1000 loss= 19.689133 acc= 0.9609
step= 1100 loss= 5.4488425 acc= 0.9663
step= 1200 loss= 1.8088338 acc= 0.969
step= 1300 loss= 12.199297 acc= 0.9713
step= 1400 loss= 2.3286042 acc= 0.9732
step= 1500 loss= 5.8512526 acc= 0.9722
step= 1600 loss= 3.3809063 acc= 0.9747
step= 1700 loss= 3.142814 acc= 0.9738
step= 1800 loss= 3.3827357 acc= 0.9765
step= 1900 loss= 6.416314 acc= 0.9749
step= 2000 loss= 2.1367373 acc= 0.978
step= 2100 loss= 2.334475 acc= 0.9768
step= 2200 loss= 1.7006538 acc= 0.9769
step= 2300 loss= 2.1535819 acc= 0.9803
step= 2400 loss= 5.64052 acc= 0.9794
step= 2500 loss= 4.7208233 acc= 0.9783
step= 2600